# Coloring B&W images 

#### Using OpenCv library for performing computer vision related operations

In [1]:
import tensorflow as tf
from tensorflow.keras import backend
backend.clear_session()
import numpy as np
import os
import random
from matplotlib import pyplot as plt

from tensorflow.keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose
from tensorflow.keras.layers import Activation, Dense, Dropout, Flatten
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

#### Converting RGB image to LAB space image

The CIELAB color space, also referred to as L*a*b* , is a color space defined by the International Commission on Illumination.t expresses color as three values: L* for perceptual lightness, and a* and b* for the four unique colors of human vision: red, green, blue, and yellow. CIELAB was intended as a perceptually uniform space, where a given numerical change corresponds to a similar perceived change in color. While the LAB space is not truly perceptually uniform, it nevertheless is useful in industry for detecting small differences in color.

In [2]:
import cv2
#load RGB image
original_img = cv2.imread('girl.jpg')
print(original_img.shape)

#convert GRAY to LAB image
lab_image = cv2.cvtColor(original_img, cv2.COLOR_RGB2LAB)
lab_image = np.array(lab_image, dtype=float)

X = lab_image[:,:,0]*(1.0/255)
Y = lab_image[:,:,1:]*(1.0/255)

Y = Y/128
X = X.reshape(1, lab_image.shape[0], lab_image.shape[1], 1)
Y = Y.reshape(1, lab_image.shape[0], lab_image.shape[1], 2)

(340, 638, 3)


In [3]:
# Building the neural network   
model = Sequential()
model.add(InputLayer(input_shape=(None, None,1)))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))

In [4]:
# Finish model
model.compile(optimizer='adam',loss='mse')

#### Increase epochs for Improving efficiency of the model

In [5]:
model.fit(x=X, 
    y=Y,
    batch_size=1,
    epochs=2000)

Epoch 2000/2000
1/1 [==============================] - 0s 251ms/step - loss: 5.9724e-08


In [6]:
print(model.evaluate(X, Y, batch_size=1))
output = model.predict(X)
output *= 128*255
X*=255
# Output colorizations
cur = np.zeros((lab_image.shape[0],lab_image.shape[1], 3), dtype=np.uint8)
cur[:,:,0] = X[0][:,:,0]
cur[:,:,1:] = output[0]

output_image = cv2.cvtColor(cur, cv2.COLOR_LAB2RGB)
Input_image = cv2.cvtColor(original_img, cv2.COLOR_RGB2GRAY)

1/1 [==============================] - 1s 540ms/step - loss: 5.4680e-08
5.4679738070717576e-08


In [7]:
cv2.imshow('Ouput', output_image)
cv2.imshow('Input', Input_image)
cv2.waitKey(0)
cv2.destroyAllWindows()